# Unified Experiment Results Analysis

This notebook provides interactive analysis of results from `unified_experiments.py`.

## Quick Start

1. Run experiments: `python experiments/unified_experiments.py --mode validation`
2. Update the `RESULTS_FILE` path below
3. Run all cells to generate comprehensive analysis

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = (12, 4)

%matplotlib inline

## 1. Load Results

In [ ]:
# UPDATE THIS PATH
RESULTS_FILE = 'results/unified_validation_20251030_123456.json'

# Load data
with open(RESULTS_FILE, 'r') as f:
    results = json.load(f)

# Filter successful runs
results = [r for r in results if r.get('status') == 'success']

# Create DataFrame
df = pd.DataFrame(results)

print(f"Loaded {len(df)} successful experiments")
print(f"Configuration space:")
print(f"  T values: {sorted(df['T'].unique())}")
print(f"  c_Q values: {sorted(df['c_Q'].unique())}")
print(f"  λ values: {sorted(df['lambda'].unique())}")
print(f"  m values: {sorted(df['m'].unique())}")
print(f"  Seeds: {sorted(df['seed'].unique())}")

In [ ]:
# Quick preview
display(df[['T', 'c_Q', 'lambda', 'm', 'test_risk', 'traj_gap', 'base_gap', 'improvement_percent']].head(10))

## 2. Summary Statistics

In [ ]:
# Overall statistics
print("=" * 70)
print("OVERALL STATISTICS")
print("=" * 70)

summary = df[['test_risk', 'traj_bound', 'base_bound', 'traj_gap', 'base_gap', 
              'improvement_percent', 'traj_kl', 'base_kl']].describe()
display(summary)

# Key findings
print("\nKey Findings:")
print(f"  Average test risk: {df['test_risk'].mean():.4f} ± {df['test_risk'].std():.4f}")
print(f"  Average trajectory gap: {df['traj_gap'].mean():.4f} ± {df['traj_gap'].std():.4f}")
print(f"  Average baseline gap: {df['base_gap'].mean():.4f} ± {df['base_gap'].std():.4f}")
print(f"  Average improvement: {df['improvement_percent'].mean():.2f}% ± {df['improvement_percent'].std():.2f}%")
print(f"  Configs with improvement > 0: {(df['improvement_percent'] > 0).sum()}/{len(df)} ({100*(df['improvement_percent'] > 0).mean():.1f}%)")

## 3. Trajectory Length Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Group by T
T_grouped = df.groupby('T').agg({
    'traj_gap': ['mean', 'std', 'min', 'max'],
    'base_gap': ['mean', 'std'],
    'improvement_percent': ['mean', 'std']
}).reset_index()

T_vals = T_grouped['T'].values

# Plot 1: Bound gaps
axes[0].errorbar(T_vals, T_grouped[('traj_gap', 'mean')], 
                yerr=T_grouped[('traj_gap', 'std')],
                marker='o', label='Trajectory', linewidth=2, capsize=5)
axes[0].errorbar(T_vals, T_grouped[('base_gap', 'mean')],
                yerr=T_grouped[('base_gap', 'std')],
                marker='s', label='Baseline', linewidth=2, capsize=5)
axes[0].set_xlabel('Trajectory Length T', fontsize=12)
axes[0].set_ylabel('Bound Gap', fontsize=12)
axes[0].set_title('Bound Tightness vs Trajectory Length', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot 2: Improvement
axes[1].errorbar(T_vals, T_grouped[('improvement_percent', 'mean')],
                yerr=T_grouped[('improvement_percent', 'std')],
                marker='D', color='green', linewidth=2, capsize=5)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='No improvement')
axes[1].set_xlabel('Trajectory Length T', fontsize=12)
axes[1].set_ylabel('Improvement (%)', fontsize=12)
axes[1].set_title('Bound Improvement vs T', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Plot 3: Distribution by T
df.boxplot(column='traj_gap', by='T', ax=axes[2])
axes[2].set_xlabel('Trajectory Length T', fontsize=12)
axes[2].set_ylabel('Trajectory Bound Gap', fontsize=12)
axes[2].set_title('Gap Distribution by T', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.savefig('analysis_trajectory_length_detailed.png', dpi=300, bbox_inches='tight')
plt.show()

# Statistical test
from scipy.stats import spearmanr
corr, pval = spearmanr(df['T'], df['traj_gap'])
print(f"\nSpearman correlation (T vs traj_gap): {corr:.3f} (p={pval:.4f})")

## 4. Posterior Scaling Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Group by c_Q
cQ_grouped = df.groupby('c_Q').agg({
    'traj_gap': ['mean', 'std'],
    'traj_kl': ['mean', 'std'],
    'trace_Sigma_Q': ['mean', 'std']
}).reset_index()

cQ_vals = cQ_grouped['c_Q'].values

# Plot 1: Bound gap
axes[0].errorbar(cQ_vals, cQ_grouped[('traj_gap', 'mean')],
                yerr=cQ_grouped[('traj_gap', 'std')],
                marker='o', linewidth=2, capsize=5)
optimal_idx = cQ_grouped[('traj_gap', 'mean')].idxmin()
axes[0].scatter([cQ_vals[optimal_idx]], [cQ_grouped[('traj_gap', 'mean')].iloc[optimal_idx]],
               s=300, marker='*', color='red', zorder=5, 
               label=f'Optimal: c_Q={cQ_vals[optimal_idx]:.2f}')
axes[0].set_xlabel('Posterior Scaling c_Q', fontsize=12)
axes[0].set_ylabel('Bound Gap', fontsize=12)
axes[0].set_title('Optimal c_Q Selection', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot 2: KL divergence
axes[1].errorbar(cQ_vals, cQ_grouped[('traj_kl', 'mean')],
                yerr=cQ_grouped[('traj_kl', 'std')],
                marker='s', color='orange', linewidth=2, capsize=5)
axes[1].set_xlabel('Posterior Scaling c_Q', fontsize=12)
axes[1].set_ylabel('KL Divergence', fontsize=12)
axes[1].set_title('KL vs c_Q', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Plot 3: Posterior variance
axes[2].errorbar(cQ_vals, cQ_grouped[('trace_Sigma_Q', 'mean')],
                yerr=cQ_grouped[('trace_Sigma_Q', 'std')],
                marker='^', color='green', linewidth=2, capsize=5)
axes[2].set_xlabel('Posterior Scaling c_Q', fontsize=12)
axes[2].set_ylabel('Trace(Σ_Q)', fontsize=12)
axes[2].set_title('Posterior Variance vs c_Q', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analysis_posterior_scaling_detailed.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nOptimal c_Q: {cQ_vals[optimal_idx]:.2f}")
print(f"  Mean gap: {cQ_grouped[('traj_gap', 'mean')].iloc[optimal_idx]:.4f}")
print(f"  Std gap: {cQ_grouped[('traj_gap', 'std')].iloc[optimal_idx]:.4f}")

## 5. Hutchinson Sample Size Analysis

In [ ]:
if len(df['m'].unique()) > 1:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    m_grouped = df.groupby('m').agg({
        'traj_gap': ['mean', 'std'],
        'trace_H_bar': ['mean', 'std']
    }).reset_index()
    
    m_vals = m_grouped['m'].values
    
    # Plot 1: Stability
    axes[0].errorbar(m_vals, m_grouped[('traj_gap', 'mean')],
                    yerr=m_grouped[('traj_gap', 'std')],
                    marker='o', linewidth=2, capsize=5)
    axes[0].set_xlabel('Hutchinson Sample Size m', fontsize=12)
    axes[0].set_ylabel('Bound Gap', fontsize=12)
    axes[0].set_title('Bound Stability vs Sample Size', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Curvature estimate
    axes[1].errorbar(m_vals, m_grouped[('trace_H_bar', 'mean')],
                    yerr=m_grouped[('trace_H_bar', 'std')],
                    marker='s', color='orange', linewidth=2, capsize=5)
    axes[1].set_xlabel('Hutchinson Sample Size m', fontsize=12)
    axes[1].set_ylabel('Trace(H̄_T)', fontsize=12)
    axes[1].set_title('Curvature Estimate vs m', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('analysis_hutchinson_detailed.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Only one m value tested, skipping Hutchinson analysis")

## 6. Heatmap: T vs c_Q

In [ ]:
# Create pivot table for heatmap
pivot = df.groupby(['T', 'c_Q'])['improvement_percent'].mean().unstack()

plt.figure(figsize=(12, 6))
sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=0, 
            cbar_kws={'label': 'Improvement (%)'})
plt.xlabel('Posterior Scaling c_Q', fontsize=12)
plt.ylabel('Trajectory Length T', fontsize=12)
plt.title('Bound Improvement Heatmap: T vs c_Q', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('analysis_heatmap_T_cQ.png', dpi=300, bbox_inches='tight')
plt.show()

# Find best configuration
best_idx = df['traj_gap'].idxmin()
best_config = df.loc[best_idx]
print("\nBest Configuration:")
print(f"  T = {best_config['T']}")
print(f"  c_Q = {best_config['c_Q']}")
print(f"  λ = {best_config['lambda']}")
print(f"  m = {best_config['m']}")
print(f"  Trajectory gap: {best_config['traj_gap']:.4f}")
print(f"  Improvement: {best_config['improvement_percent']:.1f}%")

## 7. Export Results for LaTeX

In [ ]:
# Create LaTeX-ready table
latex_df = df.groupby(['T', 'c_Q']).agg({
    'test_risk': 'mean',
    'traj_gap': ['mean', 'std'],
    'base_gap': ['mean', 'std'],
    'improvement_percent': ['mean', 'std']
}).round(4)

# Save to CSV
latex_df.to_csv('results_for_paper.csv')
print("Saved: results_for_paper.csv")

# Generate LaTeX table snippet
with open('results_table.tex', 'w') as f:
    f.write(latex_df.to_latex(escape=False))
print("Saved: results_table.tex")

## 8. Key Findings Summary

In [ ]:
print("=" * 70)
print("KEY FINDINGS SUMMARY")
print("=" * 70)

print("\n1. TRAJECTORY LENGTH EFFECT:")
T_effect = df.groupby('T')['improvement_percent'].mean()
print(f"   Avg improvement by T:")
for T, imp in T_effect.items():
    print(f"     T={T:3d}: {imp:+6.2f}%")

print("\n2. POSTERIOR SCALING EFFECT:")
cQ_effect = df.groupby('c_Q')['traj_gap'].mean()
optimal_cQ = cQ_effect.idxmin()
print(f"   Optimal c_Q: {optimal_cQ:.2f}")
print(f"   Gap at optimal: {cQ_effect[optimal_cQ]:.4f}")

print("\n3. OVERALL PERFORMANCE:")
print(f"   Trajectory tighter than baseline: {(df['traj_gap'] < df['base_gap']).sum()}/{len(df)} configs")
print(f"   Mean improvement: {df['improvement_percent'].mean():+.2f}% ± {df['improvement_percent'].std():.2f}%")
print(f"   Best improvement: {df['improvement_percent'].max():+.2f}%")
print(f"   Worst case: {df['improvement_percent'].min():+.2f}%")

print("\n4. RECOMMENDATIONS:")
if df['improvement_percent'].mean() > 0:
    print("   ✓ Trajectory-aware bounds show improvement on average")
else:
    print("   ✗ Trajectory-aware bounds do NOT improve on average")
    print("   → Check implementation or reconsider hyperparameters")

print(f"\n   Suggested config for best results:")
print(f"     T = {best_config['T']}")
print(f"     c_Q = {best_config['c_Q']}")
print(f"     λ = {best_config['lambda']}")

print("\n" + "=" * 70)